test youtube api stuff

In [1]:
import argparse
import os
import re

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow


In [35]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file("youtube_secrets.json", ["https://www.googleapis.com/auth/youtube.upload", "https://www.googleapis.com/auth/youtube"])
  credentials = flow.run_console()
  return build("youtube", "v3", credentials = credentials)

In [36]:
youtube = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=135093931493-s5i5phtkooh7rhapha96bit460us52jd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=rn08urpu6KhKSZQUcYHLWujLhWkmhN&prompt=consent&access_type=offline


In [31]:
from googleapiclient.http import MediaFileUpload


def create_video_upload_request(youtube: build, title: str, file_path: str) -> build:
    body=dict(
    snippet=dict(
      title=title
    ),
    status=dict(
      privacyStatus="unlisted"
    )
  )
    return (youtube.videos().insert(
        part=",".join(body.keys()),
        body=body,
        media_body=MediaFileUpload(file_path, chunksize=-1, resumable=True),
    )
    )

def start_video_upload(video_upload_request: build) -> str:
    response = video_upload_request.next_chunk()
    return response[1]["id"]

In [20]:
uploader = create_video_upload_request(youtube, "test_video", "output/2022-12-26.mp4")


In [21]:
response = start_video_upload(uploader)

In [32]:
video_id = response[1]["id"]

In [41]:
def add_video_to_playlist(youtube,playlist_id, video_id):
    body=dict(
        snippet=dict(
            playlistId=playlist_id,
            resourceId=dict(
                kind="youtube#video",
                videoId=video_id
            )
        )
    )
    add_video_request=youtube.playlistItems().insert(
    part="snippet",
    body=body
    ).execute()


In [42]:
add_video_to_playlist(youtube,"PLtZv6jHN_L88JZmqB7yhdxAtm3MEn3CQH",video_id)